# 날씨 데이터 결측치 채우기...  
* 애초에 해당 일자 row가 없어서 생긴 문제인 것 같네요~ ㅠㅠ

In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## 0. 데이터 로드

In [2]:
cd /

/


In [3]:
cd content/drive/MyDrive/2021/날씨 공모전/code

/content/drive/MyDrive/2021/날씨 공모전/code


In [141]:
weather = pd.read_csv('../data/weather_final.csv', encoding='cp949')

In [142]:
weather['날짜'] = weather['날짜'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
weather['year'] = weather['날짜'].apply(lambda x : x.year)
weather['month'] = weather['날짜'].apply(lambda x : x.month)

In [143]:
weather.head()

,날짜,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부,year,month
0,2018-01-01,105,1.3,5.7,-2.1,0.0,0.0,3.7,강릉,20.066667,13.400000,25.4,57.9,1023.0,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0,2018,1
1,2018-01-01,112,-0.3,2.7,-2.7,0.0,0.0,1.6,인천,37.518681,18.641758,67.2,53.8,1020.3,2018,1,1,1,0,1월1일,1,0,3,9.992379,-42.924922,0,0,2018,1
2,2018-01-01,119,-1.7,4.7,-6.9,0.0,0.0,1.0,수원,42.782895,21.375000,84.7,52.7,1025.1,2018,1,1,1,0,1월1일,1,0,3,10.136402,-45.791893,0,0,2018,1
3,2018-01-01,136,-1.0,4.7,-6.5,0.0,0.0,2.2,안동,39.750000,28.583333,56.4,58.3,1010.6,2018,1,1,1,0,1월1일,1,0,3,8.356692,-44.375704,0,0,2018,1
4,2018-01-01,152,2.1,6.2,-0.4,0.0,0.0,3.3,울산,38.548193,15.433735,42.8,58.3,1017.1,2018,1,1,1,0,1월1일,1,0,3,8.861429,-38.064334,0,0,2018,1


## 1. 결측치 확인

In [144]:
weather.isnull().sum() # PM10, PM25, 일조시간합

날짜              0
지점번호            0
평균기온            0
최고기온            0
최저기온            0
일별강수량           0
1시간최대강수량        0
평균풍속            0
지역              0
PM10          195
PM25          195
1시간최대습도         0
일조시간합           6
최고현지기압          0
연               0
월               0
일               0
분기              0
요일              0
공휴일명        11136
공휴일여부           0
주말여부            0
계절              0
체감온도            0
열지수             0
폭염여부            0
강수여부            0
year            0
month           0
dtype: int64

### 1-1. 미세먼지

#### 1-1-1. `PM10`

In [159]:
weather[weather.PM10.isnull()]['지점번호'].unique() # 안동, 제주, 천안, 강릉

array([], dtype=int64)

In [152]:
''' 해당 지역, 해당 년월의 평균치로 결측치를 대체하는 함수'''

def missing_value2(data, aws_id, year, var):
    month_list = data[(data[var].isnull())&(data.지점번호==aws_id)&(data.year==year)]['month'].unique()
    for month in month_list:
        sample = data[(data.지점번호==aws_id)&(data.year==year)&(data.month==month)&(~data[var].isnull())]
        alt = sample[var].mean() # 평균
        # 대체
        if np.isnan(alt)==True: # 해당 년,월의 변수값이 하나도 없으면 전체지역 연평균으로 대체
            data.loc[(data.지점번호==aws_id)&(data.year==year)&(data.month==month),var] = data.loc[(data.year==year, var)].mean()
        else:
            data.loc[(data.지점번호==aws_id)&(data.year==year)&(data.month==month),var] = alt 
    return data

In [153]:
# 안동
weather = missing_value2(weather, 136, 2018, 'PM10')
weather = missing_value2(weather, 136, 2019, 'PM10')

# 제주
weather = missing_value2(weather, 184, 2018, 'PM10')
weather = missing_value2(weather, 184, 2019, 'PM10')

# 천안
weather = missing_value2(weather, 232, 2018, 'PM10')
weather = missing_value2(weather, 232, 2019, 'PM10')

# 강릉 
weather = missing_value2(weather, 105, 2018, 'PM10')
weather = missing_value2(weather, 105, 2019, 'PM10')

In [154]:
weather.PM10.isnull().sum()

0

#### 1-1-2. `PM25`

In [155]:
weather[weather.PM25.isnull()]['지점번호'].unique() # 안동, 제주, 천안, 강릉

array([136, 184, 232, 105])

In [156]:
# 안동
weather = missing_value2(weather, 136, 2018, 'PM25')
weather = missing_value2(weather, 136, 2019, 'PM25')

# 제주
weather = missing_value2(weather, 184, 2018, 'PM25')
weather = missing_value2(weather, 184, 2019, 'PM25')

# 천안
weather = missing_value2(weather, 232, 2018, 'PM25')
weather = missing_value2(weather, 232, 2019, 'PM25')

# 강릉 
weather = missing_value2(weather, 105, 2018, 'PM25')
weather = missing_value2(weather, 105, 2019, 'PM25')

In [158]:
weather.PM25.isnull().sum()

0

### 1-2. 일조시간

In [160]:
weather[weather.일조시간합.isnull()]['지점번호'].unique() # 대구, 부산, 서울

array([143, 159, 108])

In [162]:
# 대구
weather = missing_value2(weather, 143, 2018, '일조시간합')
weather = missing_value2(weather, 143, 2019, '일조시간합')

# 부산 
weather = missing_value2(weather, 159, 2018, '일조시간합')
weather = missing_value2(weather, 159, 2019, '일조시간합')

# 서울
weather = missing_value2(weather, 108, 2018, '일조시간합')
weather = missing_value2(weather, 108, 2019, '일조시간합')

In [163]:
weather.일조시간합.isnull().sum()

0

## 2. 결측치 점검

In [166]:
weather.isnull().sum()

날짜              0
지점번호            0
평균기온            0
최고기온            0
최저기온            0
일별강수량           0
1시간최대강수량        0
평균풍속            0
지역              0
PM10            0
PM25            0
1시간최대습도         0
일조시간합           0
최고현지기압          0
연               0
월               0
일               0
분기              0
요일              0
공휴일명        11136
공휴일여부           0
주말여부            0
계절              0
체감온도            0
열지수             0
폭염여부            0
강수여부            0
year            0
month           0
dtype: int64

## 3. output

In [168]:
weather.to_csv('../data/weather_final2.csv', encoding='cp949', index=None)